<div style="background-color:#8B0000; border-radius:5px; border:#ffffff solid; padding: 20px; font-size:120%; text-align:center">
    <h1 align="center" style="color:#ffffff;"><b>NextFlix Cinematic Curation 🎥</b></h1>
</div>


<div style="background-color:#000000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <img src="https://media.tenor.com/NerN41mjgV0AAAAC/netflix-intro.gif" alt="Animated GIF">
</div>

<div style="background-color:#8B0000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#000000;"><b>IMPORT DEPENDENCIES</b></h2>
</div>

In [5]:
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from wordcloud import WordCloud
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.sparse import save_npz
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('netflix_data.csv')

In [7]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


<div style="background-color:#8B0000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#000000;"><b>DATA PRE-PROCESSING AND EDA</b></h2>
</div>

In [8]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

fill na with ' '

In [9]:
df.fillna('',inplace=True)

In [10]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
show_id,8807,8807,s1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
type,8807,2,Movie,6131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,8807,8807,Dick Johnson Is Dead,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
director,8807,4529,,2634,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cast,8807,7693,,825,NaN,NaN,NaN,NaN,NaN,NaN,NaN
country,8807,749,United States,2818,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date_added,8807,1768,"January 1, 2020",109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
release_year,8807.0,NaN,NaN,NaN,2014.180198,8.819312,1925.0,2013.0,2017.0,2019.0,2021.0
rating,8807,18,TV-MA,3207,NaN,NaN,NaN,NaN,NaN,NaN,NaN
duration,8807,221,1 Season,1793,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="background-color:#8B0000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#000000;"><b>Visualization</b></h2>
</div>

In [15]:
movie_counts = df['release_year'].value_counts().sort_index()
fig = px.line(x=movie_counts.index, y=movie_counts.values, labels={'x': 'Year', 'y': 'Number of Movies'},
              title='Number of Movies Released Each Year', line_shape='linear', markers=True)

fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',
    paper_bgcolor='rgb(17, 17, 17)',
    font_color='white',
)

fig.update_traces(line=dict(color='red'), marker=dict(color='red'))
fig.show()


In [ ]:
movie_type_counts = df['type'].value_counts()

fig = go.Figure(data=go.Pie(labels=movie_type_counts.index, values=movie_type_counts.values))

fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',  
    paper_bgcolor='rgb(17, 17, 17)', 
    font_color='white',  
    title='Distribution of C. Types',
)
fig.update_traces(marker=dict(colors=['red']))
fig.show()

In [19]:
movie_type_counts = df['type'].value_counts()

fig = go.Figure(data=go.Pie(labels=movie_type_counts.index, values=movie_type_counts.values))

fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',  
    paper_bgcolor='rgb(17, 17, 17)', 
    font_color='white',  
    title='Distribution of C. Types',
)
fig.update_traces(marker=dict(colors=['BLue']))
fig.show()

In [21]:
import plotly.express as px

top_countries = df['country'].value_counts().head(10)

fig = px.bar(
    x=top_countries.values,
    y=top_countries.index,
    orientation='h',
    labels={'x': 'Number of Movies', 'y': 'Country'},
    title='Top Countries with Highest Number of Movies',
    color=top_countries.values,  # You can customize the color palette
)

fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',
    paper_bgcolor='rgb(17, 17, 17)',
    font_color='white',
)

fig.show()


In [22]:
country_movie_counts = df['country'].value_counts()

data = pd.DataFrame({'Country': country_movie_counts.index, 'Movie Count': country_movie_counts.values})

fig = px.choropleth(data_frame=data, locations='Country', locationmode='country names',
                    color='Movie Count', title='Number of Movies Released By Country',
                    color_continuous_scale='Reds', range_color=(0, data['Movie Count'].max()),
                    labels={'Movie Count': 'Number of Movies'})

fig.update_layout(
    geo=dict(bgcolor='rgb(17, 17, 17)', lakecolor='rgb(17, 17, 17)'),
    plot_bgcolor='rgb(17, 17, 17)',
    paper_bgcolor='rgb(17, 17, 17)',
    font_color='white',
    coloraxis_colorbar=dict(
        title='Number of Movies',
        tickvals=[0, data['Movie Count'].max() // 2, data['Movie Count'].max()],
        ticktext=['Low', 'Medium', 'High'],
        ticks='outside'
    )
)

fig.show()


In [24]:
import plotly.express as px

ratings = list(df['rating'].value_counts().index)
rating_counts = list(df['rating'].value_counts().values)

fig = px.bar(
    y=ratings,
    x=rating_counts,
    orientation='h',
    title='Movie Ratings Distribution',
    labels={'y': 'Rating', 'x': 'Count'},
    color=ratings,
    color_discrete_sequence=px.colors.qualitative.Set2,
)

fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0.7)',
    font=dict(color='white'),
    showlegend=False,  # Hide legend for better aesthetics
)

fig.show()


In [27]:
import plotly.express as px

ratings = list(df['duration'].value_counts().index)
rating_counts = list(df['duration'].value_counts().values)

fig = px.bar(
    y=ratings,
    x=rating_counts,
    orientation='h',
    title='Movie duration Distribution',
    labels={'y': 'duration', 'x': 'Count'},
    color=ratings,
    color_discrete_sequence=px.colors.qualitative.Set2,
)

fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0.7)',
    font=dict(color='white'),
    showlegend=False,  # Hide legend for better aesthetics
)

fig.show()


<div style="background-color:#8B0000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#000000;"><b>FEATURE ENGINEERING</b></h2>
</div>

In [28]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,,,,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [29]:
new_df = df[['title', 'type', 'director', 'cast', 'rating', 'listed_in', 'description']]
new_df.set_index('title', inplace=True)

In [30]:
new_df

,type,director,cast,rating,listed_in,description
title,,,,,,
Dick Johnson Is Dead,Movie,Kirsten Johnson,,PG-13,Documentaries,"As her father nears the end of his life, filmm..."
Blood & Water,TV Show,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",TV-MA,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
Ganglands,TV Show,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",TV-MA,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
Jailbirds New Orleans,TV Show,,,TV-MA,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
Kota Factory,TV Show,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",TV-MA,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...
Zodiac,Movie,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",R,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
Zombie Dumb,TV Show,,,TV-Y7,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
Zombieland,Movie,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",R,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...


In [32]:
class TextCleaner:
    def seperate_text(self, texts):
        unique_texts = set()
        for text in texts.split(','):
            unique_texts.add(text.strip().lower())
        return ' '.join(unique_texts)
    
    def remove_space(self, texts):
        return texts.replace(' ', '').lower()
    def remove_punc(self, texts):
        texts = texts.lower()
        texts = texts.translate(str.maketrans('', '', string.punctuation))
        return ' '.join(texts.split())
    
    def clean_text(self, texts):
        texts = self.seperate_text(texts)
        texts = self.remove_space(texts)
        texts = self.remove_punc(texts)
        return texts

In [33]:
cleaner = TextCleaner()

In [35]:
new_df['type']        = new_df['type'].apply(cleaner.remove_space)
new_df['director']    = new_df['director'].apply(cleaner.seperate_text)
new_df['cast']        = new_df['cast'].apply(cleaner.seperate_text)
new_df['rating']      = new_df['rating'].apply(cleaner.remove_space)
new_df['listed_in']   = new_df['listed_in'].apply(cleaner.seperate_text)
new_df['description'] = new_df['description'].apply(cleaner.remove_punc)

In [36]:
new_df.head(1)

,type,director,cast,rating,listed_in,description
title,,,,,,
Dick Johnson Is Dead,movie,kirsten johnson,,pg-13,documentaries,as her father nears the end of his life filmma...


In [37]:
new_df['BoW'] = new_df.apply(lambda row: ' '.join(row.dropna().values),  axis=1)
new_df.drop(new_df.columns[:-1], axis=1, inplace=True)

In [38]:
new_df.head()

,BoW
title,
Dick Johnson Is Dead,movie kirsten johnson pg-13 documentaries as ...
Blood & Water,tvshow ryle de morny dillon windvogel sello m...
Ganglands,tvshow julien leclercq sofia lesaffre sami bou...
Jailbirds New Orleans,tvshow tv-ma docuseries reality tv feuds fli...
Kota Factory,tvshow urvi singh mayur more alam khan jitend...


In [39]:
tfid = TfidfVectorizer()
tfid_matrix = tfid.fit_transform(new_df['BoW'])

<div style="background-color:#8B0000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#000000;"><b>Model Building</b></h2>
</div>

In [40]:
cosine_s = cosine_similarity(tfid_matrix, tfid_matrix)

In [41]:
cosine_s

array([[1.        , 0.00504833, 0.02011193, ..., 0.01065369, 0.02109898,
        0.03048859],
       [0.00504833, 1.        , 0.01714561, ..., 0.00103121, 0.        ,
        0.00481712],
       [0.02011193, 0.01714561, 1.        , ..., 0.00560911, 0.01042642,
        0.0333502 ],
       ...,
       [0.01065369, 0.00103121, 0.00560911, ..., 1.        , 0.05649084,
        0.00600011],
       [0.02109898, 0.        , 0.01042642, ..., 0.05649084, 1.        ,
        0.01046521],
       [0.03048859, 0.00481712, 0.0333502 , ..., 0.00600011, 0.01046521,
        1.        ]])

In [42]:
np.save('tfidf_matrix.npy',tfid_matrix)
np.save('cosine_s_matrix', cosine_s)

<div style="background-color:#8B0000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#000000;"><b>Save Vectorizer</b></h2>
</div>

In [44]:
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(tfid, file)

In [45]:
final_df = df[['title','type']]

In [46]:
final_df.head()

,title,type
0,Dick Johnson Is Dead,Movie
1,Blood & Water,TV Show
2,Ganglands,TV Show
3,Jailbirds New Orleans,TV Show
4,Kota Factory,TV Show


In [47]:
final_df.to_csv('netflix_clean.csv',index=False)

<div style="background-color:#8B0000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#000000;"><b>Movie Recommendation System</b></h2>
</div>

In [58]:
import re
class FlixHub:
    def __init__(self, df, cosine_sim):
        self.df = df
        self.cosine_sim = cosine_s
    
    def recommendation(self, title, total_result=5, threshold=0.5):
        idx = self.find_id(title)
        self.df['similarity'] = self.cosine_sim[idx]
        sort_df = self.df.sort_values(by='similarity', ascending=False)[1:total_result+1]
        
        movies = sort_df['title'][sort_df['type'] == 'Movie']
        tv_shows = sort_df['title'][sort_df['type'] == 'TV Show']
        
        similar_movies = []
        similar_tv_shows = []
        
        for i, movie in enumerate(movies):
            similar_movies.append('{}. {}'.format(i+1, movie))
        
        for i, tv_show in enumerate(tv_shows):
            similar_tv_shows.append('{}. {}'.format(i+1, tv_show))
        
        return similar_movies, similar_tv_shows

    def find_id(self, name):
        for index, string in enumerate(self.df['title']):
            if re.search(name, string):
                return index
        return -1

In [63]:
flix_hub = FlixHub(final_df, cosine_s)
movies, tv_shows = flix_hub.recommendation('Money heist', total_result=10, threshold=0.5)

print('Similar Movie(s) list:')
for movie in movies:
    print(movie)

print('\nSimilar TV_show(s) list:')
for tv_show in tv_shows:
    print(tv_show)

Similar Movie(s) list:
1. Kajraare
2. Jatts in Golmaal
3. ReMastered: Devil at the Crossroads
4. Blood Money
5. ReMastered: Who Killed Jam Master Jay?
6. Krish Trish and Baltiboy: Face Your Fears
7. A Very Country Christmas
8. BluffMaster!
9. Once In A Lifetime Sessions with Moby
10. Tell Me Who I Am

Similar TV_show(s) list:
